In [ ]:
import dotenv
import os
dotenv.load_dotenv()
import csv
import json
import requests
import time
import pandas as pd
from datetime import datetime
from typing import Dict, List, Tuple
from urllib.parse import urlencode
import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
import google_bq_utils as bqu
import clickhouse_utils as ch
import opstack_metadata_utils as ops
sys.path.pop()

In [ ]:
def read_addresses_from_csv(filename: str) -> List[Tuple[str, str, str]]:
    addresses = []
    with open(filename, 'r') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # Skip header
        for row in reader:
            chain, address, name = row
            addresses.append((chain, address, name))
    return addresses

def get_balance(network: str, address: str, max_retries: int = 3, initial_wait: float = 10.0) -> float:
    # Get RPC URL from metadata
    rpc_url = ops.get_rpc_url_by_chain_name(network)
    if not rpc_url:
        print(f"No RPC URL found for network: {network}")
        return None
    
    for attempt in range(max_retries):
        # RPC request payload for eth_getBalance
        payload = {
            "jsonrpc": "2.0",
            "method": "eth_getBalance",
            "params": [address, "latest"],
            "id": 1
        }
        
        headers = {
            "Content-Type": "application/json"
        }

        try:
            response = requests.post(rpc_url, json=payload, headers=headers, timeout=30)
            
            if response.status_code == 200:
                result = response.json()
                if "result" in result and result["result"] is not None:
                    # Convert hex balance to decimal and then to ETH
                    balance_wei = int(result["result"], 16)
                    balance_eth = balance_wei / 1e18
                    return balance_eth
                else:
                    print(f"RPC returned error for {address}: {result.get('error', 'Unknown error')}")
            else:
                print(f"HTTP {response.status_code} for {address}")
                
        except Exception as e:
            print(f"Request failed for {address}: {str(e)}")
        
        # If we get here, the request failed or returned invalid data
        if attempt < max_retries - 1:  # Don't sleep on the last attempt
            wait_time = initial_wait * (2 ** attempt)  # Exponential backoff
            print(f"Attempt {attempt + 1} failed for {address}. Waiting {wait_time} seconds before retry...")
            time.sleep(wait_time)
        else:
            print(f"All {max_retries} attempts failed for {address}. Returning None.")
    
    return None

def check_balances(addresses: List[Tuple[str, str, str]]) -> Dict[str, float]:
    balances = {}
    for chain, address, name in addresses:
        print(f"Checking balance for {name} ({address}) on {chain}...")
        balance = get_balance(chain, address)
        print(f"Name: {name}, Address: {address}, Balance: {balance}")
        if balance is not None:
            balances[name] = {"balance": balance, "address": address}
        else:
            balances[name] = {"balance": None, "address": address}
        time.sleep(0.1)  # Rate limiting between different addresses
    return balances

def get_inflight_withdrawals():
        wds = d.get_dune_data(query_id = 3939869, #https://dune.com/queries/3939869
                name = "l1_to_l2_inflight_withdrawals",
                path = "outputs",
                performance="large",
                num_hours_to_rerun=1 #almost always rereun
                )
        wds_bal = wds['amount_eth'].sum()
        return wds_bal

In [ ]:
csv_filename = 'inputs/address_list.csv'  # Make sure this file exists in the same directory as your script
addresses = read_addresses_from_csv(csv_filename)
balances = check_balances(addresses)
balances['l1_to_l2_inflight_withdrawals'] = {"balance": get_inflight_withdrawals(), "address": 'https://dune.com/embeds/3939869/6626683/'} 

In [ ]:
# Output balances as JSON
json_txt = json.dumps(balances, indent=2)
# print(json_txt)
# Create DataFrame
current_time = datetime.now()
df_data = []
for name, data in balances.items():
    df_data.append({
        'dt': pd.Timestamp(current_time.date()),
        'timestamp': pd.Timestamp(current_time.isoformat()),
        'address_type': name,
        'address': data['address'],
        'balance': data['balance']
    })

df = pd.DataFrame(df_data)

In [ ]:
table_name = 'op_collective_balances'

In [ ]:
df.dtypes

In [ ]:
df.sample(5)

In [ ]:
upload_table = f'daily_{table_name}'
#BQ Upload
bqu.write_df_to_bq_table(df, f'latest_{table_name}', dataset_id='api_table_uploads', write_mode='overwrite')
bqu.append_and_upsert_df_to_bq_table(df, f'daily_{table_name}', unique_keys = ['dt','address_type','address'])
#CH Upload
ch.write_df_to_clickhouse(df, upload_table, if_exists='append')